# Using AI21 Summarize on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Summarize** using Amazon SageMaker.


--------------------
## <font color='orange'>Important:</font>
Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-dkwy6chb63hk2">https://aws.amazon.com/marketplace/pp/prodview-dkwy6chb63hk2</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.


-------------------

**AI21 Summarize** model offers access to our top-of-the-line summarization engine. It has been specifically developed for understanding long texts and providing a faithful summary of the original document. As a task-specific model, it can be easily integrated into existing systems without requiring prompt engineering: simply provide the text you want to summarize and get the best summary in the market (according to a [recent study](https://www.ai21.com/blog/summarize-api-outperforms-openais-models)).



## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Summarize a single paragraph](#B.-Summarize-a-single-paragraph)
   1. [Summarize longer texts](#C.-Summarize-longer-texts)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [ ]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/summarize-1-1-003-c51dc6a4ff7e34a1b55ac1e4f337baba"
}

In [ ]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

In [ ]:
# Import wikipedia for longer texts example
! pip install wikipedia
import wikipedia

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [ ]:
boto3.__version__

### Install ai21 python SDK

In [ ]:
! pip install -U "ai21[AWS]==1.1.4"
import ai21

In [ ]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

### <span style='color:Blue'> How to choose the best instance for my use case?</span>
<span style='color:#0057FF'> When you create your endpoint, you need to choose the instance type to run the model on. Choosing the right instance is mainly a matter of economics. Depending on your use case, you probably want the most cost-effective instance possible. In this notebook we use one of the supported instances.</span>

<span style='color:#0057FF'>Looking for the list of all supported instances? See</span> [here](https://docs.ai21.com/docs/choosing-the-right-instance-type-for-amazon-sagemaker-models#ai21-summarize).

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [ ]:
endpoint_name = "summarize"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.12xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

Once endpoint has been created, you would be able to perform real-time inference.

### B. Summarize a single paragraph

**AI21 Summarize model** offers access to our world-class summarization engine. It has been specifically developed for understanding long texts and providing a faithful summary of the original document.

This model takes a piece of text and generates summaries that remain faithful to the original document (i.e. no external information is added during the process).

The input text should contain **at least 40 words** and **no more than 50,000 characters**.

### Important - Model Usage
The input to the model should be simple and straight-forward, no prompting needed.

**Good input:** *[TEXT]*

**Bad input:** *Summarize the following text: [TEXT]*

The following is an example of a summary of a single paragraph taken from a news article.

In [ ]:
text = """The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected."""

response = ai21.Summarize.execute(
                          source=text,
                          sourceType="TEXT",
                          destination=ai21.SageMakerDestination(endpoint_name)
)

print("Original text:")
print(text)
print("================")
print("Summary:")
print(response.summary)

### C. Summarize longer texts

AI21 Summarize can handle inputs up to 50,000 characters, as stated above. This translates into roughly 10,000 words, or a whopping 40 pages.

As a demonstration of the model's behavior, we'll load a page from Wikipedia. If you're an Office fan, this will look familiar :)

In [ ]:
wiki_page = wikipedia.page("The_Office_(American_TV_series)").content

Since this is a full page, we will focus only the opening section.

In [ ]:
office_text = wiki_page.split("\n\n")[0]
print(office_text)

The summaries are formatted as bullet lists, following the original text flow.

In [ ]:
response = ai21.Summarize.execute(
                          source=office_text,
                          sourceType="TEXT",
                          destination=ai21.SageMakerDestination(endpoint_name)
)

print(response.summary)

Here is the full text and the bullet points summarized:

In [ ]:
print("Original text:")
print(office_text)
print("====================")
print("Summary:")
summaries = response.summary.split("\n")
for summary in summaries:
    print("- " + summary)

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/summarize-api) to understand all the capabilities of AI21 Summarize model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()